In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from tools.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR
import logging
import numpy as np
from argparse import ArgumentParser

In [10]:
%tb
parser = ArgumentParser(add_help=False)
# Dataset parameters
parser.add_argument('--dataset-path', type=str, default=PROCESSED_DATA_DIR / 'smallerSampleDataset')
parser.add_argument('--reshape-size', default=[224, 224, 3])
parser.add_argument('--phishing-test-size', default=0.4)
parser.add_argument('--num-targets', type=int, default=5)
parser.add_argument('--legit-imgs-num', default=420)
parser.add_argument('--phish-imgs-num', default=160)
# Model parameters
parser.add_argument('--input-shape', default=[224, 224, 3])
parser.add_argument('--margin', type=float, default=2.2)
parser.add_argument('--new-conv-params', default=[5, 5, 512])
# Training parameters
parser.add_argument('--start-lr', type=float, default=2e-5)  # 0.00002
parser.add_argument('--output-dir', type=str, default=INTERIM_DATA_DIR / 'smallerSampleDataset')
parser.add_argument('--saved-model-name', type=str, default='model')  # from first training
parser.add_argument('--new-saved-model-name', type=str, default='model2')
parser.add_argument('--save-interval', type=int, default=2000)
parser.add_argument('--batch-size', type=int, default=32)
parser.add_argument('--n-iter', type=int, default=50000)
parser.add_argument('--lr-interval', type=int, default=250)
# hard examples training
parser.add_argument('--num-sets', type=int, default=100)
parser.add_argument('--iter-per-set', type=int, default=8)
# parser.add_argument('--n_iter', type=int, default=30)


logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
)
logger = logging.getLogger()

args = parser.parse_args()

SystemExit: 2

usage: ipykernel_launcher.py [-h] [--dataset-path DATASET_PATH]
                             [--reshape-size RESHAPE_SIZE]
                             [--phishing-test-size PHISHING_TEST_SIZE]
                             [--num-targets NUM_TARGETS]
                             [--legit-imgs-num LEGIT_IMGS_NUM]
                             [--phish-imgs-num PHISH_IMGS_NUM]
                             [--input-shape INPUT_SHAPE] [--margin MARGIN]
                             [--new-conv-params NEW_CONV_PARAMS]
                             [--start-lr START_LR] [--output-dir OUTPUT_DIR]
                             [--saved-model-name SAVED_MODEL_NAME]
                             [--new-saved-model-name NEW_SAVED_MODEL_NAME]
                             [--save-interval SAVE_INTERVAL]
                             [--batch-size BATCH_SIZE] [--n-iter N_ITER]
                             [--lr-interval LR_INTERVAL] [--num-sets NUM_SETS]
                             [--iter-per-set ITER_P

SystemExit: 2

/home/jarcin/.cache/uv/archive-v0/qYiJMBX8PViqgTqT_9JOB/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Define paths for saved .npy files
imgs_train_path = args.output_dir / 'all_imgs_train.npy'
labels_train_path = args.output_dir / 'all_labels_train.npy'
file_names_train_path = args.output_dir / 'all_file_names_train.npy'

imgs_test_path = args.output_dir / 'all_imgs_test.npy'
labels_test_path = args.output_dir / 'all_labels_test.npy'
file_names_test_path = args.output_dir / 'all_file_names_test.npy'

# Initialize variables
all_imgs_train, all_labels_train, all_file_names_train = None, None, None
all_imgs_test, all_labels_test, all_file_names_test = None, None, None
data_path_trusted = args.dataset_path / 'trusted_list'
data_path_phish = args.dataset_path / 'phishing'

# Check if all .npy files exist
if (imgs_train_path.exists() and labels_train_path.exists() and file_names_train_path.exists() and
        imgs_test_path.exists() and labels_test_path.exists() and file_names_test_path.exists()):
    logger.info('Loading pre-saved data')

    # Load pre-saved data
    all_imgs_train = np.load(imgs_train_path)
    all_labels_train = np.load(labels_train_path)
    all_file_names_train = np.load(file_names_train_path)

    all_imgs_test = np.load(imgs_test_path)
    all_labels_test = np.load(labels_test_path)
    all_file_names_test = np.load(file_names_test_path)

In [ ]:
logger.info('Images loaded')

X_train_legit = all_imgs_train
y_train_legit = all_labels_train
# Load the same train/split in phase 1
phish_test_idx = np.load(args.output_dir / 'test_idx.npy')
phish_train_idx = np.load(args.output_dir / 'train_idx.npy')

X_test_phish = all_imgs_test[phish_test_idx, :]
y_test_phish = all_labels_test[phish_test_idx, :]

X_train_phish = all_imgs_test[phish_train_idx, :]
y_train_phish = all_labels_test[phish_train_idx, :]

In [7]:
from trainer_phase2 import all_targets_start_end